In [1]:
!pip install pymorphy3

In [2]:
import re

def insert_spaces(text):
    """
    Применяем набор правил, для формирования начального разбиения предложений
    """
    new_text = ""
    prev_char = ""

    # Список сокращений, которые нельзя разрывать
    abbreviations = ["т.е.", "и т.д.", "г.", "ул.", "д.", "и т.п."]

    i = 0
    while i < len(text):
        char = text[i]
        at_start = (i == 0)

        # Проверка сокращений
        matched_abbr = None
        for abbr in abbreviations:
            if text[i:i+len(abbr)] == abbr:
                matched_abbr = abbr
                break
        if matched_abbr:
            if new_text and not new_text.endswith(" "):
                new_text += " "
            new_text += matched_abbr
            i += len(matched_abbr)
            prev_char = matched_abbr[-1]
            continue

        # 1) Пробел после знаков препинания (, . ! ?), кроме троеточия в начале
        if prev_char in ",.!?":
            if not char.isspace():
                new_text += " "

        # 2) Смена языка (русский - английский)
        if prev_char and prev_char.isalpha() and char.isalpha():
            if (prev_char.isascii() and not char.isascii()) or (not prev_char.isascii() and char.isascii()):
                new_text += " "

        # 3) Большая буква - начало нового слова
        if prev_char and char.isupper() and prev_char.islower():
            new_text += " "

        # 4) Цифры отделяем от букв
        if prev_char and ((prev_char.isdigit() and char.isalpha()) or (prev_char.isalpha() and char.isdigit())):
            new_text += " "

        # 5) Скобки и кавычки
        opening_quotes = "([{«'\""
        closing_quotes = ")]}»'\""

        if char in opening_quotes and new_text and not new_text.endswith(" "):
            new_text += " "
        elif char in closing_quotes and i+1 < len(text) and not text[i+1].isspace():
            new_text += char + " "
            prev_char = char
            i += 1
            continue

        new_text += char
        prev_char = char
        i += 1

    # 6) Убираем лишние пробелы перед знаками препинания
    new_text = re.sub(r"\s+([,.!?])", r"\1", new_text)
    # 7) Убираем лишние пробелы в начале и конце
    new_text = new_text.strip()

    return new_text

# Примеры использования
text1 = "онсказал:'Яволк'родилсявСССР"
text2 = "Яродлсяв2014,этопосле2013года."

print(insert_spaces(text1))
print(insert_spaces(text2))


онсказал: 'Яволк 'родилсяв СССР
Яродлсяв 2014, этопосле 2013 года.


In [3]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

def is_known_word(word):
    """
    Проверяем знает ли наш анализатор такое слово
    """
    one_letter_words = ['а', 'б', 'в', 'и', 'к', 'о', 'c', 'у', 'я'] # однобуквенные слова
    if len(word) == 1 and word not in one_letter_words:
        return False
    parses = morph.parse(word.lower())
    return any(p.is_known for p in parses)

def dp_split_longest_word(text, max_word_len=30):
    """
    DP с приоритетом:
    1) минимальное количество слов
    2) среди разбиений с одинаковым количеством слов — максимально длинные слова
    """
    n = len(text)
    dp = [(float('inf'), [], [])] * (n + 1)
    dp[0] = (0, [], [])

    for i in range(1, n + 1):
        best = (float('inf'), [], [])
        found_known = False
        for j in range(max(0, i - max_word_len), i):
            candidate = text[j:i]
            if is_known_word(candidate):
                found_known = True
                prev_num, prev_words, prev_lengths = dp[j]
                num_words = prev_num + 1
                lengths = prev_lengths + [len(candidate)]
                if (num_words < best[0]) or (num_words == best[0] and lengths > best[2]):
                    best = (num_words, prev_words + [candidate], lengths)
        if found_known:
            dp[i] = best
        else:
            j = i - 1
            while j >= 0 and dp[j][0] == float('inf'):
                j -= 1
            unknown_segment = text[j:i] if j >= 0 else text[:i]
            num_words = (dp[j][0] if j >= 0 else 0) + 1
            words = (dp[j][1] if j >= 0 else []) + [unknown_segment]
            lengths = (dp[j][2] if j >= 0 else []) + [len(unknown_segment)]
            dp[i] = (num_words, words, lengths)

    return dp[n][1]


In [4]:
def merge_single_letters(words):
    """
    Объединяет подряд идущие одиночные буквы в одно слово,
    кроме 'я'
    """
    result = []
    buffer = []

    for w in words:
        if len(w) == 1 and w != 'я':
            buffer.append(w)
        else:
            if buffer:
                result.append("".join(buffer))
                buffer = []
            result.append(w)

    if buffer:  # добавляем оставшиеся в буфере
        result.append("".join(buffer))

    return result

In [5]:
import string

def process_text(text):
  """
    Финальная функция применяющая все шаги алгоритма:
    1. Первичное деление по правилам
    2. Более умное дробление с дп
    3. Конкатенция слишком раздробленных слов (однобуквенных)
  """

  text = insert_spaces(text)

  splited = text.split()
  new_split = []

  for token in splited:
    if token[0].isdigit():
        new_split.append(token)
        continue
    if re.match(r"^[A-Za-z]+$", token[0]):
        new_split.append(token)
        continue
    start = ''
    for i in token:
      if i not in string.punctuation:
        break
      start += i

    clean_token = token.strip(string.punctuation)
    dp_split = dp_split_longest_word(clean_token)

    end = ''
    for i in token[::-1]:
      if i not in string.punctuation:
        break
      end = i + end

    dp_split = " ".join(merge_single_letters(dp_split))
    dp_split = start + dp_split + end
    new_split.append(dp_split)

  return " ".join(new_split)



In [6]:
import pandas as pd

def read_test_file(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        next(f)
        for line in f:
            row_id, text = line.strip().split(",", 1)
            rows.append((row_id, text))
    return pd.DataFrame(rows, columns=["id", "text_no_spaces"])

def get_space_positions(original_text, spaced_text):
    positions = []
    idx = 0
    for ch in spaced_text:
        if ch == " ":
            positions.append(idx)
        else:
            idx += 1
    return positions


def process_file(input_path, output_path):
    """
    Формируем финальный файл с предсказаниями
    """
    df = read_test_file(input_path)

    ids = []
    texts = []
    positions = []

    for _, row in df.iterrows():
        row_id = row["id"]
        text = row["text_no_spaces"]

        processed_text = process_text(text)

        space_positions = get_space_positions(text, processed_text)

        ids.append(row_id)
        texts.append(processed_text)
        positions.append(space_positions)

    df = pd.DataFrame({"predicted_positions": positions})

    df = df.reset_index(drop=False).rename({"index": "id"}, axis=1)
    df.to_csv(output_path)



process_file("test.txt", "submission.csv")
